# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [3]:
%pip install numpy pandas xlsxwriter requests
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np
import pandas as pd # panel datas, tabular data (rows, cols)
# most commonly used for dataframe
import requests # gold standard for making http requests
import xlsxwriter
import math
import datetime
import time
from dotenv import load_dotenv

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [73]:
stocks = pd.read_csv('sp_500_stocks.csv')
type(stocks)
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [5]:
import os
load_dotenv()  # Load environment variables from .env file

IEX_CLOUD_API_TOKEN = os.getenv('IEX_KEY')
FZ_API_KEY = os.getenv('FZ_KEY')



## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [ ]:
symbol = 'AAPL'

api_url = f"https://api.finazon.io/latest/finazon/us_stocks_essential/time_series"
querystring = {
    'ticker':'AAPL',
    'interval': '1d',
    'start_at':str(int(time.mktime(datetime.date(2022,1,1).timetuple()))),
    'end_at':str(int(time.mktime(datetime.date(2024,1,1).timetuple()))),
    'page_size': 730 # 501 for two years minus non trading days (weekends and holidays)
}

api_key = f"apikey {FZ_API_KEY}"

data = requests.get(api_url, headers={'Authorization':api_key}, params=querystring).json()['data']

print(len(data))

data = [round((x['l'] + x['h']) / 2,2) for x in data]
data

In [82]:
# cant access iex
# api_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol}?token={IEX_CLOUD_API_TOKEN}'
# type(data)
# data.status_code

# can access finazon
# get stock value for each stock (aapl, tsla, goog)
url = "https://api.finazon.io/latest/finazon/us_stocks_essential/price"
api_key = f"apikey {FZ_API_KEY}"
headers = {"Authorization": api_key}

tickers = ['AAPL', 'TSLA', 'GOOG']
prices = {}

for t in tickers:
    querystring = {"ticker":t}

    response = requests.get(url, headers=headers, params=querystring)
    
    p = response.json()['p']

    prices[t] = [p]

prices['AAPL'].append(3_440_000_000)
prices['TSLA'].append(692_810_000)
prices['GOOG'].append(2_070_000_000)

print(prices)

{'AAPL': [226.88, 3440000000], 'TSLA': [220.8217, 692810000], 'GOOG': [168.96, 2070000000]}


## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [83]:
cols = ['ticker','price','market cap','# shares to buy']
df = pd.DataFrame(columns=cols)
df

,ticker,price,market cap,# shares to buy


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [84]:
import random
random.seed(10)
# price 1 -> 200
# market cap 1 million -> 3 trillion
for stock in stocks['Ticker']:
    price = random.randint(100,20000)/100
    mcap = random.randint(1000000,3000000000)
    prices[stock] = [price, mcap]



In [85]:
for k,v in prices.items():
    df = pd.concat(
        [
            pd.DataFrame([[
                k,
                v[0],
                v[1],
                'n/a'
            ]], columns=cols), 
            df
        ],
        ignore_index=True
    )
df

/var/folders/jr/mhy81gpn45955yh7jff8xr_c0000gr/T/ipykernel_53847/3449703307.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(


,ticker,price,market cap,# shares to buy
0,ZTS,90.7400,454543750,n/a
1,ZION,143.9700,1380821237,n/a
2,ZBRA,75.9700,1665188036,n/a
3,ZBH,155.1000,785035767,n/a
4,YUM,191.6700,2782327234,n/a
...,...,...,...,...
501,AAL,141.5300,2073586604,n/a
502,A,188.2300,140951793,n/a
503,GOOG,171.9800,2662773064,n/a
504,TSLA,220.8217,692810000,n/a


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [94]:
portflio = float(10000000)
position_size=portflio/len(df.index)
position_size # how much $ of each stock you are buying

19762.845849802372

In [95]:
for i in range(len(df.index)):
    df.loc[i, '# shares to buy'] = math.floor(position_size/df.loc[i,'price'])
df

,ticker,price,market cap,# shares to buy
0,ZTS,90.7400,454543750,217
1,ZION,143.9700,1380821237,137
2,ZBRA,75.9700,1665188036,260
3,ZBH,155.1000,785035767,127
4,YUM,191.6700,2782327234,103
...,...,...,...,...
501,AAL,141.5300,2073586604,139
502,A,188.2300,140951793,104
503,GOOG,171.9800,2662773064,114
504,TSLA,220.8217,692810000,89


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy: